# Buildind a model for the Kaggle Titanic Competition: summary (a 1st approach)

You can find the competion here: https://www.kaggle.com/c/titanic

The procedure could be the following.

## Load datasets

Load the train and test dataset. Then you will
- Train and validade on the train dataset
- Do prediction on test dataset and submit them to kaggle 

In [ ]:
import pandas as pd

train_df = pd.read_csv("data/titanic_train.csv")
test_df = pd.read_csv("data/titanic_test.csv")

Take a glimpse at the data 

In [ ]:
train_df.head()  

In [ ]:
test_df.head()  

Keep the test's _PassengerId_ for later (submission fase)

In [ ]:
test_passenger_id = test_df["PassengerId"]

Get the target from train_df (test does not have it) and drop that column

In [ ]:
target = train_df["Survived"].values
train_df = train_df.drop(["Survived"], axis=1)

Concatenate the test and train dataframes as they will suffer the same operations, but remember their dimension as you'll need them later

In [ ]:
m_train, n_train = train_df.shape
m_test, n_test = test_df.shape

df = pd.concat([train_df, test_df], axis=0)
df.info()

## Remove non-informative (?) columns

In a first approach, we might suppose that the _PassengerId_, _Name_ and _Ticket_ might be irrelevant and can be dropped

In [ ]:
df = df.drop(["PassengerId", "Name", "Ticket"], axis=1)

## Replace `nan` in numerical columns by the median/mean value of the remaining values

Replace the "nan" _Ages_ and _Fare_ by the mean value 

In [ ]:
age_mean_value = df["Age"].median()
df["Age"] = df["Age"].fillna(age_mean_value)

fare_mean_value = df["Fare"].mean()
df["Fare"] = df["Fare"].fillna(fare_mean_value)

## Apply one hot encoding to categorical classes

In our case, perhaps is better to replace the _Cabin_ by the corresponding deck (see 19-Missing-Data.ipynb) and then do the one hot encoding

In [ ]:
def transf(x):
    try:
        return x[0]
    except:
        return x

df["Cabin"] = df["Cabin"].apply(transf)
df = pd.get_dummies(df)

In [ ]:
df.head()

## Transformed dataset

The transformed dataset has now the following configuration

In [ ]:
df.info()

In [ ]:
df.describe().T

## Recover the train and test dataset

In [ ]:
train_df = df.iloc[:m_train, :]
test_df = df.iloc[m_train:, :]

## Model 

To build the model get the data from the dataframe

In [ ]:
data = train_df.values

Find the "best" `RandomForestClassifier` model doing grid search cross validation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

parameters = {"n_estimators": [10, 50, 100], "max_depth": [5, 10, 20, None]}

gscv = GridSearchCV(
    estimator=RandomForestClassifier(random_state=0),
    param_grid=parameters,
    cv=5,  # number of folds in a (Stratified)KFold,
)

gscv.fit(data, target)

## Get the best achieved model and predict for test data

In [ ]:
model = gscv.best_estimator_
prediction = model.predict(test_df.values)
prediction

## Submit to Kaggle

Build the csv file to be subbmited to Kaggle

In [ ]:
df_pred = pd.DataFrame(
    {
        "PassengerId": test_passenger_id, # retrieved after loading
        "Survived": prediction
    })

df_pred

save the prediction in a csv file

In [ ]:
df_pred.to_csv("my_titanic_survival_predictions.csv", index=False)

## Not good enough...?
If you don't like your results you can see training details and try to some actions...

In [ ]:
gscv.best_score_

The best paramters

In [ ]:
gscv.best_params_

and the CV results

In [ ]:
gscv.cv_results_